# Vehicle Route Planning

## Packages

In [1]:
## Data Acess and Manipulation
import json
import numpy as np
import pandas as pd
import snowflake.connector as sf

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## for vehicle route planning
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

## Utilities
import os
import logging
from functools import partial
#from google.colab import userdata
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor as Executor

## Geospatial Work
import folium
from folium import plugins
import osmnx as ox
import networkx as nx
from shapely.geometry import shape, Point
from math import sin, cos, sqrt, atan2, radians

## Multiprocessing
from multiprocessing import Pool, cpu_count

## Utilities
### importing necessary functions from dotenv library and loading variables from .env file
import os
from dotenv import load_dotenv 
load_dotenv()

# Geopandas
import geopandas

from tqdm import tqdm

import plotly.express as px

In [2]:
# Notebook Configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
#>> Contains layout and callbacks for app 1
''' 
Notes:
This file is for creating app 1, it ontains layout and callbacks for app 1
'''
# Import packages---------------------------------------------------------------------------------------------
## Data Acess and Manipulation
import numpy as np

## Import App Packages
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
from datetime import datetime as dt
from dash.exceptions import PreventUpdate
import pandas as pd
import io
from flask import send_file

## Data Visualization
import plotly.graph_objs as go

##Local Imports
from src.utils.data_loaders import get_data_from_sf, get_geojson, load_dict
from src.utils.queries import data_query_order_line, data_query_item_weights
from src.utils.data_processing import data_preprocessing, calculate_metrics, generate_vehicle_names, data_preprocessing_map
from src.utils.map import maps, route_maps
from src.utils.vehicle_route_planning import DistanceCalculator, create_data_model, run_model
from src.utils.routes_processing import enhance_optimized_route

from app import app

## Utilities
import os
from dotenv import load_dotenv 
load_dotenv()

# Incorporate data---------------------------------------------------------------------------------------------

True

In [4]:
df_weights = get_data_from_sf(data_query_item_weights)

In [5]:
#df_t = get_data_from_sf(data_query)
df_order_line = get_data_from_sf(data_query_order_line)

In [6]:
# = df_t.copy()

In [7]:
df_order_line['itemTotalWeight'] = df_order_line['itemTotalWeight'].replace(0, 1)
df_order_line = df_order_line[df_order_line['customerCode']!='17658'].copy() 

js = get_geojson(os.getenv("geojson_loc_2"))

In [8]:
# Preprocessing the Data
df = data_preprocessing(df_order_line, df_weights, js)

In [9]:
choropleth_df, order_details = data_preprocessing_map(df)

In [10]:
# Distance Matrics
bbox = (22.4969475367, 26.055464179, 51.5795186705, 56.3968473651)  # Set only if using 'GRAPH' method
distance_calculator = DistanceCalculator(bbox=bbox, network_type="drive")
id, coordinates, demand, dm = distance_calculator.df_to_dm(order_details, 'customerLat', 'customerLong', 'customerCode', 'totalWeight', 'base', 'GD')


In [11]:
# Solve the CVRP problem
data = create_data_model(demand, dm)
manager, routing, solution, output= run_model(data)

In [12]:
from src.utils.routes_processing import enhance_optimized_route
route_output = enhance_optimized_route(output, coordinates, id)

KeyboardInterrupt: 